# Generating images using Amazon Titan Image Generator (JavaScript)

> ☝️ This notebook should work well with the __`Data Science 3.0`__ kernel in Amazon SageMaker Studio and with the __`conda_python3`__ in a Amazon SageMaker Notebook Instance.

In the [Generating images using Amazon Titan Image Generator](https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/05_Image/bedrock-titan-image-generator.ipynb) tutorial, we have showed how to use the new [Amazon Titan Image Generator](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-image-models.html) on [Amazon Bedrock](https://aws.amazon.com/bedrock/) model to generate (text-to-image) and edit (image-to-image) images.

In this tutorial, we will show the same concepts but using JavaScript instead of Python. We highly recommend to go through [Generating images using Amazon Titan Image Generator](https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/05_Image/bedrock-titan-image-generator.ipynb) first to understand the basic concepts.

## Setup

⚠️ ⚠️ ⚠️Before running this notebook, make sure you've installed [Node.js](https://nodejs.org/en).⚠️ ⚠️ ⚠️ I followed [this blog](https://www.cherryservers.com/blog/install-nodejs-on-ubuntu) to install the 20.10.0 LTS version as of Jan 2024.

Firstly, run the below command to add the NodeSource repository.

In [ ]:
!curl -fsSL https://deb.nodesource.com/setup_20.x | bash -

Once the repository is added, install Node.js and npm by running the below command.

In [ ]:
!apt install -y nodejs

Check node version and npm version.

In [ ]:
# Should be starting with v20.xx.x.
!node -v

In [ ]:
# Should be starting with 10.x.x.
!npm -v

[Install AWS SDK for JavaScript v3](https://docs.aws.amazon.com/AWSJavaScriptSDK/v3/latest/client/bedrock-runtime/).

In [ ]:
!npm install @aws-sdk/client-bedrock-runtime

Create 2 directories for later usage.

In [ ]:
import os

In [ ]:
os.makedirs("data/js-code", exist_ok=True)
os.makedirs("data/titan-js", exist_ok=True)

## Use Cases

### Text to Image

In text-to-image mode, we provide a text description (prompt) of the image that **should** be generated.

What if we want to **avoid** specific content or stylistic choices? Because image generation models are typically trained from image descriptions, trying to directly specify what you **don't** want in the prompt (e.g. man without a beard) doesn't usually work well: it would be very unusual to describe an image by what it is not!

In the case of Amazon Titan Image Generator, we can specify a negative prompt to steer the model away from unwanted elements.

Write the below JS code to a file.

In [ ]:
%%writefile data/js-code/image_generation.js

const { BedrockRuntime } = require('@aws-sdk/client-bedrock-runtime');
const fs = require('fs');

const prompt = 'a dog walking down an urban street';
const negativePrompts = 'cars';
const outputImageNamePrefix = 'data/titan-js/image-generation';

async function run() {
  // Create a new BedrockRuntime client.
  const bedrockRuntime = new BedrockRuntime({});

  // Configure the inference parameters.
  const inferenceParams = {
    taskType: 'TEXT_IMAGE',
    textToImageParams: {
      text: prompt,
      negativeText: negativePrompts 
    },
    imageGenerationConfig: {
      numberOfImages: 1,
      quality: 'premium',
      height: 512,
      width: 512,
      cfgScale: 8.0,
      seed: 0 // Use a fixed seed for repeatability.
              // Or you can use a random seed: Math.round(Math.random() * 100000)
    }
  };

  // Invoke the model.
  const response = await bedrockRuntime.invokeModel({
    modelId: 'amazon.titan-image-generator-v1',
    contentType: 'application/json',
    accept: 'application/json',
    body: JSON.stringify(inferenceParams)
  });

  // Convert the JSON string to an object.
  const responseBody = JSON.parse(response.body.transformToString());

  // Loop through the generated images and save each to disk.
  const images = responseBody.images;
  images.forEach((image, index) => {
    const outputFileName = `${outputImageNamePrefix}-${index + 1}.png`;
    fs.writeFileSync(outputFileName, image, 'base64');
    console.log(`Generated image: ${outputFileName}.`);
  });
}

run();

Execute the js code.

In [ ]:
!node data/js-code/image_generation.js

Check the generated image.

In [ ]:
from PIL import Image

In [ ]:
Image.open("data/titan-js/image-generation-1.png")

### Image Variation

Generating images from text is powerful but, in some cases, you will need many rounds of prompt refinement to get just the right image.

Rather than starting from scratch, image-to-image generation lets us **modify** an existing image to make specific changes.

Write the below JS code to a file.

In [ ]:
%%writefile data/js-code/image_variation.js

const { BedrockRuntime } = require('@aws-sdk/client-bedrock-runtime');
const fs = require('fs');

const input_image = 'images/woman-in-gallery.jpg';
const prompt = 'a woman standing in an art gallery';
const outputImageNamePrefix = 'data/titan-js/image-variation';

console.log(`Original image: ${input_image}.`);

async function run() {
  // Create a new BedrockRuntime client.
  const bedrockRuntime = new BedrockRuntime({});

  // Load the input image from disk.
  // IMPORTANT: The image width and height must both be 1024 pixels or less.
  const inputImage = fs.readFileSync(input_image);
  const inputImageBase64 = inputImage.toString('base64');

  // Configure the inference parameters.
  const inferenceParams = {
    taskType: 'IMAGE_VARIATION',
    imageVariationParams: {
      images: [inputImageBase64],
      text: prompt // Description of the original image
    },
    imageGenerationConfig: {
      numberOfImages: 1, // Number of variations to generate
      quality: 'premium', // Allowed values are 'standard' or 'premium'
      height: 1024,
      width: 1024,
      cfgScale: 8.0,
      seed: 0 // Use a fixed seed for repeatability.
              // Or you can use a random seed: Math.round(Math.random() * 100000)
    }
  };

  // Invoke the model.
  const response = await bedrockRuntime.invokeModel({
    modelId: 'amazon.titan-image-generator-v1',
    contentType: 'application/json',
    accept: 'application/json',
    body: JSON.stringify(inferenceParams)
  });

  // Convert the JSON string to an object.
  const responseBody = JSON.parse(response.body.transformToString());

  // Loop through the generated images and save each to disk.
  const images = responseBody.images;
  images.forEach((image, index) => {
    const outputFileName = `${outputImageNamePrefix}-${index + 1}.png`;
    fs.writeFileSync(outputFileName, image, 'base64');
    console.log(`Generated image: ${outputFileName}.`);
  });
}

run();

Execute the js code.

In [ ]:
!node data/js-code/image_variation.js

Check the original image.

In [ ]:
Image.open("images/woman-in-gallery.jpg")

Check the generated image.

In [ ]:
Image.open("data/titan-js/image-variation-1.png")

### Inpainting

Another way to modify images is by using **inpainting**.

Inpainting refers to the process of replacing a portion of an image with another image based on a textual prompt.

By providing a mask image that outlines the portion to be replaced, a textual prompt, and the original image, the model can produce a new image that replaces the masked area with the object, subject, or environment described in the textual prompt.

#### Add an object to an image using mask image

Write the below JS code to a file.

In [ ]:
%%writefile data/js-code/object_addition.js

const { BedrockRuntime } = require('@aws-sdk/client-bedrock-runtime');
const fs = require('fs');

const prompt = 'an old wooden stool near the table';
const input_image = 'images/three_pots.jpg';

// TODO: Use of the inverted mask is a hack to workaround a bug in the Titan
// Image Generator API. Remove this when the bug is fixed.
// const mask_image = 'images/three_pots-add_mask.png';
const mask_image = 'images/three_pots-add_mask_INVERTED.png';

const outputImageNamePrefix = 'data/titan-js/image-addtion-mask-image';

console.log(`Original image: ${input_image}.`);
console.log(`Mask image: ${mask_image}.`);

async function run() {
  // Create a new BedrockRuntime client.
  const bedrockRuntime = new BedrockRuntime({});

  // Load the input image from disk.
  // IMPORTANT: The image width and height must both be 1024 pixels or less.
  const inputImage = fs.readFileSync(input_image);
  const inputImageBase64 = inputImage.toString('base64');

  // Load the mask image from disk. White pixels in the mask represent the
  // the area in which to generate new content.
  // IMPORTANT: The mask must be in PNG format and must contain only pure
  // black and pure white pixels.
  const maskImage = fs.readFileSync(mask_image);
  const maskImageBase64 = maskImage.toString('base64');

  // Configure the inference parameters.
  const inferenceParams = {
    taskType: 'INPAINTING',
    inPaintingParams: {
      image: inputImageBase64,
      maskImage: maskImageBase64,
      text: prompt // Add a stool to the image
    },
    imageGenerationConfig: {
      numberOfImages: 1,
      quality: 'premium', // Allowed values are 'standard' and 'premium'
      height: 1024,
      width: 1024,
      cfgScale: 8.0,
      seed: 3 // Change the seed to generate different content
    }
  };

  // Invoke the model.
  const response = await bedrockRuntime.invokeModel({
    modelId: 'amazon.titan-image-generator-v1',
    contentType: 'application/json',
    accept: 'application/json',
    body: JSON.stringify(inferenceParams)
  });

  // Convert the JSON string to an object.
  const responseBody = JSON.parse(response.body.transformToString());

  // Loop through the generated images and save each to disk.
  const images = responseBody.images;
  images.forEach((image, index) => {
    const outputFileName = `${outputImageNamePrefix}-${index + 1}.png`;
    fs.writeFileSync(outputFileName, image, 'base64');
    console.log(`Generated image: ${outputFileName}.`);
  });
}

run();

Execute the js code.

In [ ]:
!node data/js-code/object_addition.js

Check the original image.

In [ ]:
Image.open("images/three_pots.jpg")

Check the mask image.

In [ ]:
Image.open("images/three_pots-add_mask_INVERTED.png")

Check the generated image.

In [ ]:
Image.open("data/titan-js/image-addtion-mask-image-1.png")

#### Remove an object from an image using mask image

Write the below JS code to a file.

In [ ]:
%%writefile data/js-code/object_removal_using_a_mask_image.js

const { BedrockRuntime } = require('@aws-sdk/client-bedrock-runtime');
const fs = require('fs');

const input_image = 'images/three_pots.jpg';

// TODO: Use of the inverted mask is a hack to workaround a bug in the Titan
// Image Generator API. Remove this when the bug is fixed.
// const mask_image = 'images/three_pots-remove_mask.png';
const mask_image = 'images/three_pots-center_pot_mask_INVERTED.png';

const outputImageNamePrefix = 'data/titan-js/object-removal-mask-image';

console.log(`Original image: ${input_image}.`);
console.log(`Mask image: ${mask_image}.`);

async function run() {
  // Create a new BedrockRuntime client.
  const bedrockRuntime = new BedrockRuntime({});

  // Load the input image from disk.
  // IMPORTANT: The image width and height must both be 1024 pixels or less.
  const inputImage = fs.readFileSync(input_image);
  const inputImageBase64 = inputImage.toString('base64');

  // Load the mask image from disk. White pixels in the mask represent the
  // object(s) to be removed.
  // IMPORTANT: The mask must be in PNG format and must contain only pure
  // black and pure white pixels.
  const maskImage = fs.readFileSync(mask_image);
  const maskImageBase64 = maskImage.toString('base64');

  // Configure the inference parameters.
  const inferenceParams = {
    taskType: 'INPAINTING',
    inPaintingParams: {
      image: inputImageBase64,
      maskImage: maskImageBase64
      // We intentionally omit the "text" param. By doing so, the generated
      // content will attempt to match the background.
    },
    imageGenerationConfig: {
      numberOfImages: 1,
      quality: 'premium', // Allowed values are 'standard' and 'premium'
      height: 1024,
      width: 1024,
      cfgScale: 8.0,
      seed: 1 // If you don't get a clean removal you may want to try a different seed.
    }
  };

  // Invoke the model.
  const response = await bedrockRuntime.invokeModel({
    modelId: 'amazon.titan-image-generator-v1',
    contentType: 'application/json',
    accept: 'application/json',
    body: JSON.stringify(inferenceParams)
  });

  // Convert the JSON string to an object.
  const responseBody = JSON.parse(response.body.transformToString());

  // Loop through the generated images and save each to disk.
  const images = responseBody.images;
  images.forEach((image, index) => {
    const outputFileName = `${outputImageNamePrefix}-${index + 1}.png`;
    fs.writeFileSync(outputFileName, image, 'base64');
    console.log(`Generated image: ${outputFileName}.`);
  });
}

run();

In [ ]:
!node data/js-code/object_removal_using_a_mask_image.js

Check original image.

In [ ]:
Image.open("images/three_pots.jpg")

Check mask image.

In [ ]:
Image.open("images/three_pots-center_pot_mask_INVERTED.png")

Check generated image.

In [ ]:
Image.open("data/titan-js/object-removal-mask-image-1.png")

#### Replace object using a mask prompt

Write the below JS code to a file.

In [ ]:
%%writefile data/js-code/object_replacement_using_a_mask_prompt.js

const { BedrockRuntime } = require('@aws-sdk/client-bedrock-runtime');
const fs = require('fs');

const prompt = 'a pale blue potting table with drawers';
const maskPrompt = 'table';
const input_image = 'images/three_pots.jpg';
const outputImageNamePrefix = 'data/titan-js/object-replacement-mask-prompt';

console.log(`Original image: ${input_image}.`);

async function run() {
  // Create a new BedrockRuntime client.
  const bedrockRuntime = new BedrockRuntime({});

  // Load the input image from disk.
  // IMPORTANT: The image width and height must both be 1024 pixels or less.
  const inputImage = fs.readFileSync(input_image);
  const inputImageBase64 = inputImage.toString('base64');

  // Configure the inference parameters.
  const inferenceParams = {
    taskType: 'INPAINTING',
    inPaintingParams: {
      image: inputImageBase64,
      maskPrompt: maskPrompt,
      text: prompt
    },
    imageGenerationConfig: {
      numberOfImages: 1,
      quality: 'premium',
      height: 1024,
      width: 1024,
      cfgScale: 8.0,
      seed: 2 // Change the seed to generate different content
    }
  };

  // Invoke the model.
  const response = await bedrockRuntime.invokeModel({
    modelId: 'amazon.titan-image-generator-v1',
    contentType: 'application/json',
    accept: 'application/json',
    body: JSON.stringify(inferenceParams)
  });

  // Convert the JSON string to an object.
  const responseBody = JSON.parse(response.body.transformToString());

  // Loop through the generated images and save each to disk.
  const images = responseBody.images;
  images.forEach((image, index) => {
    const outputFileName = `${outputImageNamePrefix}-${index + 1}.png`;
    fs.writeFileSync(outputFileName, image, 'base64');
    console.log(`Generated image: ${outputFileName}.`);
  });
}

run();

In [ ]:
!node data/js-code/object_replacement_using_a_mask_prompt.js

Check original image

In [ ]:
Image.open("images/three_pots.jpg")

Check generated image.

In [ ]:
Image.open("data/titan-js/object-replacement-mask-prompt-1.png")

#### Remove object using a mask prompt

Write the below JS code to a file.

In [ ]:
%%writefile data/js-code/object_removal_using_a_mask_prompt.js

const { BedrockRuntime } = require('@aws-sdk/client-bedrock-runtime');
const fs = require('fs');

const maskPrompt = 'flowers in pots';
const input_image = 'images/three_pots.jpg';
const outputImageNamePrefix = 'data/titan-js/object-removal-mask-prompt';

console.log(`Original image: ${input_image}`);

async function run() {
  // Create a new BedrockRuntime client.
  const bedrockRuntime = new BedrockRuntime({});

  // Load the input image from disk.
  // IMPORTANT: The image width and height must both be 1024 pixels or less.
  const inputImage = fs.readFileSync(input_image);
  const inputImageBase64 = inputImage.toString('base64');

  // Configure the inference parameters.
  const inferenceParams = {
    taskType: 'INPAINTING',
    inPaintingParams: {
      image: inputImageBase64,
      maskPrompt: maskPrompt
      // We intentionally omit the "text" param. By doing so, the generated
      // content will attempt to match the background.
    },
    imageGenerationConfig: {
      numberOfImages: 1,
      quality: 'premium', // Allowed values are 'standard' and 'premium'
      height: 1024,
      width: 1024,
      cfgScale: 8.0,
      seed: 0 // Use a fixed seed for repeatability.
              // Or you can use a random seed: Math.round(Math.random() * 100000)
    }
  };

  // Invoke the model.
  const response = await bedrockRuntime.invokeModel({
    modelId: 'amazon.titan-image-generator-v1',
    contentType: 'application/json',
    accept: 'application/json',
    body: JSON.stringify(inferenceParams)
  });

  // Convert the JSON string to an object.
  const responseBody = JSON.parse(response.body.transformToString());

  // Loop through the generated images and save each to disk.
  const images = responseBody.images;
  images.forEach((image, index) => {
    const outputFileName = `${outputImageNamePrefix}-${index + 1}.png`;
    fs.writeFileSync(outputFileName, image, 'base64');
    console.log(`Generated image: ${outputFileName}`);
  });
}

run();

In [ ]:
!node data/js-code/object_removal_using_a_mask_prompt.js

Check original image.

In [ ]:
Image.open("images/three_pots.jpg")

Check generated image.

In [ ]:
Image.open("data/titan-js/object-removal-mask-prompt-1.png")

### Outpainting

In this final section, we are going to **extend** the image.

This process, known as **outpainting**, involves generating new pixels that seamlessly extend an image's existing boundaries.

We can do this by providing the original image and a segmentation mask, which can either be an image or a prompt.

#### Replace background using a mask prompt

Write the below JS code to a file.

In [ ]:
%%writefile data/js-code/background_replacement_using_a_mask_prompt.js

const { BedrockRuntime } = require('@aws-sdk/client-bedrock-runtime');
const fs = require('fs');

const prompt = 'woman standing by a warm fireplace';
const maskPrompt = 'person';
const input_image = 'images/woman-in-gallery.jpg';
const outputImageNamePrefix = 'data/titan-js/background-replacement-mask-prompt';

console.log(`Original image: ${input_image}`);

async function run() {
  // Create a new BedrockRuntime client.
  const bedrockRuntime = new BedrockRuntime({});

  // Load the input image from disk.
  // IMPORTANT: The image width and height must both be 1024 pixels or less.
  const inputImage = fs.readFileSync(input_image);
  const inputImageBase64 = inputImage.toString('base64');

  // Configure the inference parameters.
  const inferenceParams = {
    taskType: 'OUTPAINTING',
    outPaintingParams: {
      image: inputImageBase64,
      text: prompt, // Description of the background to generate
      maskPrompt: maskPrompt, // The element(s) to keep
      outPaintingMode: 'PRECISE' // 'DEFAULT' softens the mask. 'PRECISE' keeps it sharp.
    },
    imageGenerationConfig: {
      numberOfImages: 1, // Number of variations to generate
      quality: 'premium', // Allowed values are 'standard' and 'premium'
      height: 1024,
      width: 1024,
      cfgScale: 8.0,
      seed: Math.round(Math.random() * 100000) // Use a random seed
    }
  };

  // Invoke the model.
  const response = await bedrockRuntime.invokeModel({
    modelId: 'amazon.titan-image-generator-v1',
    contentType: 'application/json',
    accept: 'application/json',
    body: JSON.stringify(inferenceParams)
  });

  // Convert the JSON string to an object.
  const responseBody = JSON.parse(response.body.transformToString());

  // Loop through the generated images and save each to disk.
  const images = responseBody.images;
  images.forEach((image, index) => {
    const outputFileName = `${outputImageNamePrefix}-${index + 1}.png`;
    fs.writeFileSync(outputFileName, image, 'base64');
    console.log(`Generated image: ${outputFileName}.`);
  });
}

run();

In [ ]:
!node data/js-code/background_replacement_using_a_mask_prompt.js

Check original image.

In [ ]:
Image.open("images/woman-in-gallery.jpg")

Check generated image.

In [ ]:
Image.open("data/titan-js/background-replacement-mask-prompt-1.png")

#### Replace background using a mask image

Write the below JS code to a file.

In [ ]:
%%writefile data/js-code/background_replacement_using_a_mask_image.js

const { BedrockRuntime } = require('@aws-sdk/client-bedrock-runtime');
const fs = require('fs');

const prompt = 'potted flower sitting on a kitchen counter';
const input_image = 'images/three_pots.jpg';
const mask_image = 'images/three_pots-center_pot_mask_INVERTED.png';
const outputImageNamePrefix = 'data/titan-js/background-replacement-mask-image';

console.log(`Original image: ${input_image}`);
console.log(`Mask image: ${mask_image}`);

async function run() {
  // Create a new BedrockRuntime client.
  const bedrockRuntime = new BedrockRuntime({});

  // Load the input image from disk.
  // IMPORTANT: The image width and height must both be 1024 pixels or less.
  const inputImage = fs.readFileSync(input_image);
  const inputImageBase64 = inputImage.toString('base64');

  // Load the mask image from disk. Black pixels in the mask represent the
  // areas you would like to replace.
  // IMPORTANT: The mask must be in PNG format and must contain only pure
  // black and pure white pixels.
  const maskImage = fs.readFileSync(mask_image);
  const maskImageBase64 = maskImage.toString('base64');

  // Configure the inference parameters.
  const inferenceParams = {
    taskType: 'OUTPAINTING',
    outPaintingParams: {
      image: inputImageBase64,
      text: prompt, // Description of the background to generate
      maskImage: maskImageBase64,
      outPaintingMode: 'DEFAULT' // 'DEFAULT' softens the mask. 'PRECISE' keeps it sharp.
    },
    imageGenerationConfig: {
      numberOfImages: 1, // Number of variations to generate
      quality: 'premium', // Allowed values are 'standard' and 'premium'
      height: 1024,
      width: 1024,
      cfgScale: 8.0,
      seed: 0
    }
  };

  // Invoke the model.
  const response = await bedrockRuntime.invokeModel({
    modelId: 'amazon.titan-image-generator-v1',
    contentType: 'application/json',
    accept: 'application/json',
    body: JSON.stringify(inferenceParams)
  });

  // Convert the JSON string to an object.
  const responseBody = JSON.parse(response.body.transformToString());

  // Loop through the generated images and save each to disk.
  const images = responseBody.images;
  images.forEach((image, index) => {
    const outputFileName = `${outputImageNamePrefix}-${index + 1}.png`;
    fs.writeFileSync(outputFileName, image, 'base64');
    console.log(`Generated image: ${outputFileName}.`);
  });
}

run();

In [ ]:
!node data/js-code/background_replacement_using_a_mask_image.js

Check original image.

In [ ]:
Image.open("images/three_pots.jpg")

Check masked image.

In [ ]:
Image.open("images/three_pots-center_pot_mask_INVERTED.png")

Check generated image.

In [ ]:
Image.open("data/titan-js/background-replacement-mask-image-1.png")